抽样是数据处理的一种基本方法，常常伴随着计算资源不足、获取全部数据困难、时效性要求等情况使用。

抽样方法
+ 随机抽样 直接从整体数据中等概率抽取n个样本。这种方法优势是，简单、好操作、适用于分布均匀的场景；缺点是总体大时无法一一编号
+ 系统抽样 又称机械、等距抽样，将总体中个体按顺序进行编号，然后计算出间隔，再按照抽样间隔抽取个体。优势,易于理解、简便易行。缺点是，如有明显分布规律时容易产生偏差。
+ 群体抽样 总体分群，在随机抽取几个小群代表总体。优点是简单易行、便与组织；缺点是群体划分容易造成误差
+ 分层抽样 先按照观察指标影响较大的某一种特征，将总体分若干个类别，再从每一层随机抽取一定数量的单位合并成总体。优点样本代表性好，少误差

以上四种基本抽样方法都属单阶段抽样，实际应用中常根据实际情况将整个抽样过程分为若干阶段来进行，称为多阶段抽样。
各种抽样方法的抽样误差一般是：整群抽样≥单纯随机抽样≥系统抽样≥分层抽样

数据运营过程中，数据要能正确反映运营背景
数据抽样过程中要注意一些问题
+ 数据时效性 不能用过时的数据来分析现在的运营状态
+ 关键因素数据 整体数据的关键性数据必须要在模型中，如双十一带来的销售增长
+ 业务随机性 抽样数据要使各个场景的数据分布均衡
+ 数据来源多样性 数据覆盖要全面

抽样数据量问题
+ 时间分布 能包含业务周期。月销售预测，至少包含12个月数据；时间还要考虑季节、节假日、特定促销日等周期性。
+ 做预测分析 考虑特征数据和特征值域的分布，通常数据记录要同时是特征数量和特征值域的100倍以上。例如数据集有5个特征值，每个特征有2个值域，那么数据记录数需要至少1000（100*5*2）条以上
+ 做关联规则分析 根据关联前后项数量（每个前项或后项可包含多个要关联的主体，例如品牌+商品+价格关联），每个主体需要至少1000条数据。例如只做单品销售关联，那么单品的销售记录需要在1000条以上；如果要同时做单品+品牌的关联，那么需要至少2000条数据。
+ 异常检测 无论是监督室还是非监督式建模，对于异常数据本来就是小概率分布的，因此异常数据记录一般越多越好。

In [39]:
import random
import numpy as np
import pandas as pd

# 导入数据
df = pd.read_csv('https://raw.githubusercontent.com/ffzs/dataset/master/glass.csv')

df.index.size

214

In [4]:
##########随机抽样##########
#
# 使用pandas
# DataFrame.sample(n=None, frac=None, replace=False, weights=None, random_state=None, axis=None)
# n是要抽取的行数。（例如n=20000时，抽取其中的2W行）
# frac是抽取的比列。（有一些时候，我们并对具体抽取的行数不关系，我们想抽取其中的百分比，这个时候就可以选择使用frac，例如frac=0.8，就是抽取其中80%）
# replace：是否为有放回抽样，取replace=True时为有放回抽样。
# weights这个是每个样本的权重，具体可以看官方文档说明。
# random_state这个在之前的文章已经介绍过了。
# axis是选择抽取数据的行还是列。axis=0的时是抽取行，axis=1时是抽取列（也就是说axis=1时，在列中随机抽取n列，在axis=0时，在行中随机抽取n行）

df_0 = df.sample(n=20, replace=True)
df_0.index.size

20

In [40]:
data = df.values

# 使用random
data_sample = random.sample(list(data), 20)
len(data_sample)

20

In [41]:
##########等距抽样##########
# 指定抽样数量
sample_count = 50
# 获取最大样本量
record_count = data.shape[0]
# 抽样间距
width = record_count//sample_count
data_sample = []
i = 0
# 本量小于等于指定抽样数量并且矩阵索引在有效范围内是
while len(data_sample) <= sample_count and i * width <= record_count -1:
    data_sample.append(data[i*width])
    i += 1
len(data_sample)

4


51

In [35]:
##########分层抽样##########
# 数据只是随便找的分层仅限于演示
# 定义每个分层的抽样数量
each_sample_count = 6
# 定义分层值域
label_data_unique = np.unique(data[:, -1])
# 定义一些数据
sample_list, sample_data, sample_dict = [], [], {}
# 遍历每个分层标签
for label_data in label_data_unique:
    for data_tmp in data:  # 读取每条数据
        if data_tmp[-1] == label_data:
            sample_list.append(data_tmp)
    # 对每层数据都数据抽样
    each_sample_data = random.sample(sample_list, each_sample_count)
    sample_data.extend(each_sample_data)
    sample_dict[label_data] = len(each_sample_data)
sample_dict

{1.0: 6, 2.0: 6, 3.0: 6, 5.0: 6, 6.0: 6, 7.0: 6}

In [38]:
##########整群抽样##########
# 数据分群仅限于演示，不符合实际情况
# 定义整群的标签
label_data_unique = np.unique(data[:, -1])
# 随机抽取2个群
sample_label = random.sample(list(label_data_unique), 2)
# 定义空列表
sample_data = []
# 遍历每个整群标签值域
for each_label in sample_label:
    for data_tmp in data:
        if data_tmp[-1] == each_label:
            sample_data.append(data_tmp)
len(sample_data)

83